In [9]:
import json
import os
import re
import spacy

In [10]:
def get_num_participants_file(path, num):
    filenames = os.listdir(path)
    num_files = []
    for file_ in filenames:
        participants = 0
        if "." not in file_:
            continue
        with open(path+file_, "r") as f:
            for line in f.readlines():
                if "参加者の関係" in line:
                    if participants == num:
                        num_files.append(file_)
                        break
                if "参加者" in line:
                    participants += 1
    num_files.sort() 
    return num_files

In [11]:
def txt2json(path, file_, participants):

    # for file_ in plist:
    print(file_)
    file_n = file_.split(".")[0]
    conv = {
            "data":file_n,
            "turns":[]
    }
    users = {}
    with open(path+file_, "r") as f:
        conv_data_all = f.readlines()
        for line in conv_data_all:
            if "参加者の関係" in line:
                break
            if "参加者" in line:
                    user = line.split("：")[0][4:]
                    users[user] = "user{0}".format(len(users))
        # ここから互いの発話
        # 変なやつが紛れてる場合がある(data007)
        i = 0
        while True:
            conv_data = conv_data_all[4+participants+i:]
                
            line = conv_data[0]
                
            user_name = line.split("：")[0]
            # print(user_name)
            
            if user_name not in users.keys():
                print("init key error", user_name)
                i += 1
            else:
                user = users[user_name]
                utterrance = line.split("：")[1].strip()
                break
            
        for line in conv_data[1:]:
            # 話者確定
            if "：" in line:
                splited = line.split("：")
                user_name = splited[0]
                # 変なヤツ引いたら一旦飛ばす
                if user_name not in users.keys():
                    print("key error", user_name)
                    continue

                # これまでのデータを保存
                turn_data = {
                    "user" : user,
                    "utterance" : clean_utterance(utterrance)
                }
                conv["turns"].append(turn_data)

                    # 話者情報初期化
                
                utterrance = splited[1].strip()
                
                user = users[user_name]
                # 改行で連続発話
            else:
                utterrance += line.strip()
    
    return conv

In [12]:
# 発話を整形したい
nlp =  spacy.load('ja_ginza')
def clean_utterance(utt):
    doc = nlp(utt)
    valid_utt = ""
    is_continue_parren = False
    is_continue_moved = False
    for token in doc:
        if "＜" in token.orth_ or "（" in token.orth_:
            is_continue_parren = True
            continue
        
        if "＞" in token.orth_ or "）" in token.orth_:
            is_continue_parren = False
            continue
        
        # 括弧が優先
        if is_continue_parren:
            continue

        if "感動詞" in token.tag_:
            is_continue_moved = True
            continue

        if is_continue_moved and "補助記号-読点" in token.tag_:
            is_continue_moved = False
            continue
            
        if is_continue_moved:
            continue
        
        valid_utt += token.orth_
    return valid_utt
    
    

In [13]:
nucc_path = "../../corpus/nucc/law/"
participants = 2
num_files = get_num_participants_file(nucc_path, participants)
filepath = "conv{0}/".format(participants)
output = "../../corpus/nucc/" + filepath

In [16]:
# clean_utterance("あー、半強制やろ（笑い）＜名詞＞。行きたくない、もー。")
doc = nlp("あー、半強制やろ（笑）＜名詞＞。行きたくない、もー。")
for token in doc:
    print(token.orth_)

あー
、
半
強制
やろ
(
笑
)
＜
名詞
＞
。
行き
たく
ない
、
もー
。


In [85]:
completed = len( os.listdir(output) )
print("completed:", completed)
for file_ in num_files[completed:]:
    # print(file_)
    conv = txt2json(nucc_path, file_, participants)
    file_n = file_.split(".")[0]
    with open(output+"{0}.json".format(file_n), "w") as f:
        json.dump(conv, f, ensure_ascii=False, indent=4)

completed: 91


In [3]:
import numpy as np
nlp =  spacy.load('ja_ginza')
emb_dim = nlp("形態素").vector.shape[0]

def _in(arg1, arg2):
    result = False
    if isinstance(arg1, list):
        for element in arg1:
            if isinstance(element, str):
                if element in arg2:
                    result = True
                    break
            else:
                if element == arg2:
                    result = True
                    break
    else:
        if isinstance(arg1, str):
            if arg1 in arg2:
                result = True
        else:
            if arg1 == arg2:
                result = True
    return result


def filter_pos_vector(pos_list, text):
    vec = np.zeros( (emb_dim) )
    doc = nlp(text, disable=['ner'])
    for token in doc:
        # print(token, token.tag_)
        if "助動詞" not in token.tag_ and _in(pos_list, token.tag_):
            # print(token, token.tag_)
            vec += token.vector
    return vec

In [4]:
# 5文連続で有効そうな対話を取得

def extract_continue_convs(pos_list, conv, length):
    new_convs = []
    continue_conv = []
    for turn in conv:
        utterance = turn["utterance"]
        # print(utterance)
        vecter = filter_pos_vector(pos_list, utterance)
        # 有効そうな発話だったら
        mo_len = len(nlp(utterance)) 
        if mo_len > 5 and mo_len <= 30 and np.linalg.norm(vecter) > 0:
            continue_conv.append(utterance)
        else:
            continue_conv  = []
        
        if len(continue_conv) >= length:
            new_convs.append( continue_conv[-length:] )
            # print(continue_conv[-length:])
    
    return new_convs
        

In [5]:
pos_list = "名詞 代名詞 動詞 形容詞 副詞 接続詞 感動詞 連体詞".split()

In [6]:
def load_json(path, file_):
    with open(path+file_, "r") as f:
        conv = json.load(f)
    return conv


In [14]:
files = os.listdir(output)
files.sort()

length = 2
conv_json = {
    "length" : length,
    "convs" : []
}
from tqdm import tqdm
for file_ in tqdm( files ) :
    conv = load_json(output, file_)

    new_conv = extract_continue_convs(pos_list, conv["turns"],  length)
    conv_json["convs"].append(new_conv)

100%|██████████| 91/91 [09:07<00:00,  6.02s/it]


In [15]:
cont_dir = "continueV2/"
cont_path = "../../corpus/nucc/" + cont_dir
cont_path

'../../corpus/nucc/continue/'

In [16]:
with open(cont_path+"cont{0}.json".format(length), "w") as f:
    json.dump(conv_json, f, ensure_ascii=False, indent=4)